In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Wczytanie modelu YOLO
model = YOLO('yolov8n.pt')  # Model lekki, można użyć np. 'yolov8m.pt' dla lepszej dokładności

# Ścieżki do plików wejściowego i wyjściowego
input_path = 'F://BurzaVSZryw_mini.mp4'
output_path = 'test.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 2704
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 900
fps = cap.get(cv2.CAP_PROP_FPS)

# Obliczenie rozmiaru kadru dla proporcji 16:9
crop_height = height  # 900
crop_width = int(crop_height * 16 / 9)  # 1600

# Inicjalizacja zapisu wideo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (crop_width, crop_height))

# Parametry wygładzania pozycji kadru
window_size = 30  # Liczba klatek do średniej kroczącej
positions = []

def calculate_center(detections):
    """Oblicza centrum akcji na podstawie wykrytych obiektów."""
    if not detections:
        return width // 2  # Domyślnie środek, jeśli brak detekcji
    centroids = []
    for det in detections:
        x1, y1, x2, y2 = det[:4]
        centroids.append((x1 + x2) / 2)  # Pozycja X środka bounding boxa
    if centroids:
        return sum(centroids) / len(centroids)
    return width // 2

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detekcja obiektów w klatce
    results = model(frame)
    detections = results[0].boxes.data.cpu().numpy()  # Format: [x1, y1, x2, y2, confidence, class]
    
    # Filtrowanie detekcji: zawodnicy (class 0 - 'person') i piłka (class 32 - 'sports ball')
    players = [det for det in detections if int(det[5]) == 0]
    balls = [det for det in detections if int(det[5]) == 32]
    
    # Obliczenie centrum akcji
    if balls:
        # Jeśli wykryto piłkę, użyj jej pozycji
        center_x = calculate_center(balls)
    else:
        # W przeciwnym razie użyj centroidu zawodników
        center_x = calculate_center(players)
    
    # Wygładzenie pozycji kadru
    positions.append(center_x)
    if len(positions) > window_size:
        positions.pop(0)
    smoothed_center_x = sum(positions) / len(positions)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    right = left + crop_width
    
    # Przycięcie klatki
    cropped_frame = frame[:, left:right]
    
    # Zapis przyciętej klatki
    out.write(cropped_frame)

# Zwolnienie zasobów
cap.release()
out.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# Wczytanie modelu YOLO
model = YOLO('yolov8n.pt')  # Można użyć 'yolov8m.pt' dla lepszej dokładności

# Ścieżki do plików wejściowego i wyjściowego
input_path = 'F://BurzaVSZryw_mini.mp4'
output_path = 'test.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Docelowy rozmiar kadru (16:9, wysokość 720)
crop_height = 720
crop_width = int(crop_height * 16 / 9)  # 1280

# Parametry wygładzania
window_size = 15  # Zwiększone okno dla płynniejszego ruchu
positions_x = []
positions_y = []
smoothing_factor = 0.1  # Waga dla wygładzania liniowego

# Inicjalizacja zapisu wideo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (crop_width, crop_height))

def split_frame(frame, tile_size=640):
    """Dzieli obraz na kafelki dla lepszej detekcji małych obiektów."""
    tiles = []
    h, w = frame.shape[:2]
    for y in range(0, h, tile_size):
        for x in range(0, w, tile_size):
            tile = frame[y:min(y + tile_size, h), x:min(x + tile_size, w)]
            tiles.append((tile, x, y))
    return tiles

def calculate_center(detections, frame_width, frame_height):
    """Oblicza centrum akcji, priorytetyzując piłkę."""
    if not detections:
        return frame_width // 2, frame_height // 2
    balls = [det for det in detections if int(det[5]) == 32]  # 'sports ball'
    players = [det for det in detections if int(det[5]) == 0]  # 'person'
    
    if balls:
        # Priorytet dla piłki
        x1, y1, x2, y2 = balls[0][:4]
        return (x1 + x2) / 2, (y1 + y2) / 2
    elif players:
        # Średnia pozycja zawodników
        centroids_x = [(det[0] + det[2]) / 2 for det in players]
        centroids_y = [(det[1] + det[3]) / 2 for det in players]
        return sum(centroids_x) / len(centroids_x), sum(centroids_y) / len(centroids_y)
    return frame_width // 2, frame_height // 2

prev_center_x, prev_center_y = width // 2, height // 2
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Podział na kafelki dla detekcji
    tiles = split_frame(frame, tile_size=640)
    detections = []
    for tile, tile_x, tile_y in tiles:
        results = model(tile, classes=[0, 32])  # Ogranicz do 'person' i 'sports ball'
        tile_detections = results[0].boxes.data.cpu().numpy()
        # Przeskalowanie współrzędnych do oryginalnego obrazu
        for det in tile_detections:
            det[0] += tile_x  # x1
            det[2] += tile_x  # x2
            det[1] += tile_y  # y1
            det[3] += tile_y  # y2
            detections.append(det)
    
    # Obliczenie centrum akcji
    center_x, center_y = calculate_center(detections, width, height)
    
    # Wygładzanie liniowe
    center_x = prev_center_x * (1 - smoothing_factor) + center_x * smoothing_factor
    center_y = prev_center_y * (1 - smoothing_factor) + center_y * smoothing_factor
    prev_center_x, prev_center_y = center_x, center_y
    
    # Wygładzanie średnią kroczącą
    positions_x.append(center_x)
    positions_y.append(center_y)
    if len(positions_x) > window_size:
        positions_x.pop(0)
        positions_y.pop(0)
    smoothed_center_x = sum(positions_x) / len(positions_x)
    smoothed_center_y = sum(positions_y) / len(positions_y)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    top = int(max(0, min(smoothed_center_y - crop_height / 2, height - crop_height)))
    right = left + crop_width
    bottom = top + crop_height
    
    # Przycięcie klatki
    cropped_frame = frame[top:bottom, left:right]
    
    # Skalowanie do 1280x720, jeśli kadr nie jest dokładny
    if cropped_frame.shape != (crop_height, crop_width, 3):
        cropped_frame = cv2.resize(cropped_frame, (crop_width, crop_height), interpolation=cv2.INTER_AREA)
    
    # Zapis klatki
    out.write(cropped_frame)

# Zwolnienie zasobów
cap.release()
out.release()
cv2.destroyAllWindows()


0: 640x640 1 person, 102.9ms
Speed: 4.2ms preprocess, 102.9ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 10 persons, 76.5ms
Speed: 3.1ms preprocess, 76.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 10 persons, 68.8ms
Speed: 3.5ms preprocess, 68.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 62.2ms
Speed: 3.1ms preprocess, 62.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x160 (no detections), 50.0ms
Speed: 1.2ms preprocess, 50.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 160)

0: 288x640 (no detections), 71.5ms
Speed: 1.4ms preprocess, 71.5ms inference, 0.8ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 (no detections), 43.9ms
Speed: 1.8ms preprocess, 43.9ms inference, 0.8ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 (no detections), 40.1ms
Speed: 1.0ms preprocess, 40.1ms inference, 0.8ms postp

In [3]:
import torch
print(torch.cuda.is_available())  # Powinno zwrócić True
print(torch.cuda.get_device_name(0))  # Nazwa GPU

False


AssertionError: Torch not compiled with CUDA enabled